In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/h-1b-visa/h1b_kaggle.csv


In [3]:
df = pd.read_csv("../input/h-1b-visa/h1b_kaggle.csv")

In [4]:
df.shape

(3002458, 11)

In [5]:
df.head()

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,1,CERTIFIED-WITHDRAWN,UNIVERSITY OF MICHIGAN,BIOCHEMISTS AND BIOPHYSICISTS,POSTDOCTORAL RESEARCH FELLOW,N,36067.0,2016.0,"ANN ARBOR, MICHIGAN",-83.743038,42.280826
1,2,CERTIFIED-WITHDRAWN,"GOODMAN NETWORKS, INC.",CHIEF EXECUTIVES,CHIEF OPERATING OFFICER,Y,242674.0,2016.0,"PLANO, TEXAS",-96.698886,33.019843
2,3,CERTIFIED-WITHDRAWN,"PORTS AMERICA GROUP, INC.",CHIEF EXECUTIVES,CHIEF PROCESS OFFICER,Y,193066.0,2016.0,"JERSEY CITY, NEW JERSEY",-74.077642,40.728158
3,4,CERTIFIED-WITHDRAWN,"GATES CORPORATION, A WHOLLY-OWNED SUBSIDIARY O...",CHIEF EXECUTIVES,"REGIONAL PRESIDEN, AMERICAS",Y,220314.0,2016.0,"DENVER, COLORADO",-104.990251,39.739236
4,5,WITHDRAWN,PEABODY INVESTMENTS CORP.,CHIEF EXECUTIVES,PRESIDENT MONGOLIA AND INDIA,Y,157518.4,2016.0,"ST. LOUIS, MISSOURI",-90.199404,38.627003


**Handle Missing Data**

In [6]:
df.isnull().any()

Unnamed: 0            False
CASE_STATUS            True
EMPLOYER_NAME          True
SOC_NAME               True
JOB_TITLE              True
FULL_TIME_POSITION     True
PREVAILING_WAGE        True
YEAR                   True
WORKSITE              False
lon                    True
lat                    True
dtype: bool

In [7]:
df.isnull().sum()

Unnamed: 0                 0
CASE_STATUS               13
EMPLOYER_NAME             59
SOC_NAME               17734
JOB_TITLE                 43
FULL_TIME_POSITION        15
PREVAILING_WAGE           85
YEAR                      13
WORKSITE                   0
lon                   107242
lat                   107242
dtype: int64

In [8]:
df = df.dropna(axis=0, subset=['CASE_STATUS'])

In [9]:
df = df.drop(['Unnamed: 0', 'EMPLOYER_NAME', 'JOB_TITLE','WORKSITE', 'lon','lat'], axis = 1)

In [10]:
df['SOC_NAME'] = df['SOC_NAME'].fillna(df['SOC_NAME'].mode()[0])

In [11]:
df['FULL_TIME_POSITION'] = df['FULL_TIME_POSITION'].fillna(df['FULL_TIME_POSITION'].mode()[0])

In [12]:
df['PREVAILING_WAGE'] = df['PREVAILING_WAGE'].fillna(df['PREVAILING_WAGE'].mode()[0])

In [13]:
df.isnull().sum()

CASE_STATUS           0
SOC_NAME              0
FULL_TIME_POSITION    0
PREVAILING_WAGE       0
YEAR                  0
dtype: int64

In [14]:
df = df[df['PREVAILING_WAGE'] <= 500000]

In [15]:
df.CASE_STATUS.value_counts() 

CERTIFIED                                             2615530
CERTIFIED-WITHDRAWN                                    202650
DENIED                                                  92202
WITHDRAWN                                               89584
PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED         15
REJECTED                                                    2
INVALIDATED                                                 1
Name: CASE_STATUS, dtype: int64

In [16]:
df.head()

,CASE_STATUS,SOC_NAME,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR
0,CERTIFIED-WITHDRAWN,BIOCHEMISTS AND BIOPHYSICISTS,N,36067.0,2016.0
1,CERTIFIED-WITHDRAWN,CHIEF EXECUTIVES,Y,242674.0,2016.0
2,CERTIFIED-WITHDRAWN,CHIEF EXECUTIVES,Y,193066.0,2016.0
3,CERTIFIED-WITHDRAWN,CHIEF EXECUTIVES,Y,220314.0,2016.0
4,WITHDRAWN,CHIEF EXECUTIVES,Y,157518.4,2016.0


In [17]:
df.shape

(2999984, 5)

In [18]:
s0 = df.CASE_STATUS[df.CASE_STATUS.eq("CERTIFIED")].sample(85000).index
s1 = df.CASE_STATUS[df.CASE_STATUS.eq("CERTIFIED-WITHDRAWN")].sample(85000).index 
s2 = df.CASE_STATUS[df.CASE_STATUS.eq("DENIED")].sample(85000).index
s3 = df.CASE_STATUS[df.CASE_STATUS.eq("WITHDRAWN")].sample(85000).index 
# s4 = df.CASE_STATUS[df.CASE_STATUS.eq("PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED")].sample(15).index 
# s5 = df.CASE_STATUS[df.CASE_STATUS.eq("REJECTED")].sample(2).index 
# s6 = df.CASE_STATUS[df.CASE_STATUS.eq("INVALIDATED")].sample(1).index 
# samp = df.loc[s0.union(s1).union(s2).union(s3).union(s4).union(s5).union(s6)]
samp = df.loc[s0.union(s1).union(s2).union(s3)]

In [19]:
samp.shape

(340000, 5)

In [20]:
samp.head()

,CASE_STATUS,SOC_NAME,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR
1,CERTIFIED-WITHDRAWN,CHIEF EXECUTIVES,Y,242674.0,2016.0
3,CERTIFIED-WITHDRAWN,CHIEF EXECUTIVES,Y,220314.0,2016.0
4,WITHDRAWN,CHIEF EXECUTIVES,Y,157518.4,2016.0
11,CERTIFIED-WITHDRAWN,CHIEF EXECUTIVES,Y,163717.0,2016.0
12,CERTIFIED-WITHDRAWN,CHIEF EXECUTIVES,Y,203860.8,2016.0


In [21]:
samp.CASE_STATUS.value_counts() 

DENIED                 85000
WITHDRAWN              85000
CERTIFIED              85000
CERTIFIED-WITHDRAWN    85000
Name: CASE_STATUS, dtype: int64

In [22]:
df = samp

In [23]:
import sys
df['SOC_NAME1'] = 'others'
df['SOC_NAME1'][df['SOC_NAME'].str.contains('COMPUTER','SOFTWARE')] = 'it'
df['SOC_NAME1'][df['SOC_NAME'].str.contains('CHIEF','MANAGEMENT')] = 'manager'
df['SOC_NAME1'][df['SOC_NAME'].str.contains('MECHANICAL')] = 'mechanical'
df['SOC_NAME1'][df['SOC_NAME'].str.contains('DATABASE')] = 'database'
df['SOC_NAME1'][df['SOC_NAME'].str.contains('SALES','MARKET')] = 'scm'
df['SOC_NAME1'][df['SOC_NAME'].str.contains('FINANCIAL')] = 'finance'
df['SOC_NAME1'][df['SOC_NAME'].str.contains('PUBLIC','FUNDRAISING')] = 'pr'
df['SOC_NAME1'][df['SOC_NAME'].str.contains('EDUCATION','LAW')] = 'administrative'
df['SOC_NAME1'][df['SOC_NAME'].str.contains('AUDITORS','COMPLIANCE')] = 'audit'
df['SOC_NAME1'][df['SOC_NAME'].str.contains('DISTRIBUTION','LOGISTICS')] = 'scm'
df['SOC_NAME1'][df['SOC_NAME'].str.contains('RECRUITERS','HUMAN','human','recruiters')] = 'hr'
df['SOC_NAME1'][df['SOC_NAME'].str.contains('AGRICULTURAL','FARM')] = 'agri'
df['SOC_NAME1'][df['SOC_NAME'].str.contains('CONSTRUCTION','ARCHITECTURAL')] = 'estate'
df['SOC_NAME1'][df['SOC_NAME'].str.contains('FORENCSIC','HEALTH','health','forencsic')] = 'medical'
df['SOC_NAME1'][df['SOC_NAME'].str.contains('TEACHERS','teachers')] = 'education'

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

TypeError: unsupported operand type(s) for &: 'str' and 'int'

In [24]:
df.SOC_NAME.value_counts() 

Computer Systems Analysts                           26874
Computer Programmers                                22819
SOFTWARE DEVELOPERS, APPLICATIONS                   22231
Software Developers, Applications                   19922
COMPUTER SYSTEMS ANALYSTS                           18618
                                                    ...  
PROJECT MANAGER                                         1
TAX EXAMINERS AND COLLECTORS, AND REVENUE AGENTS        1
COST ESTIMATOR                                          1
Driver/Sales Workers                                    1
ELEMENTARY SCHOOL TEACHERS, EXCEPT SPECIAL ED.          1
Name: SOC_NAME, Length: 1525, dtype: int64

In [25]:
df.SOC_NAME1.value_counts() 

others            276008
it                 52320
finance             3331
mechanical          2189
audit               2137
database            1667
scm                 1061
administrative       627
pr                   442
manager              218
Name: SOC_NAME1, dtype: int64

In [26]:
df.head()

,CASE_STATUS,SOC_NAME,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,SOC_NAME1
1,CERTIFIED-WITHDRAWN,CHIEF EXECUTIVES,Y,242674.0,2016.0,manager
3,CERTIFIED-WITHDRAWN,CHIEF EXECUTIVES,Y,220314.0,2016.0,manager
4,WITHDRAWN,CHIEF EXECUTIVES,Y,157518.4,2016.0,manager
11,CERTIFIED-WITHDRAWN,CHIEF EXECUTIVES,Y,163717.0,2016.0,manager
12,CERTIFIED-WITHDRAWN,CHIEF EXECUTIVES,Y,203860.8,2016.0,manager


In [27]:
df['SOC_NAME'] = df['SOC_NAME1']
df = df.drop(['SOC_NAME1'], axis = 1)

In [28]:
df.head()

,CASE_STATUS,SOC_NAME,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR
1,CERTIFIED-WITHDRAWN,manager,Y,242674.0,2016.0
3,CERTIFIED-WITHDRAWN,manager,Y,220314.0,2016.0
4,WITHDRAWN,manager,Y,157518.4,2016.0
11,CERTIFIED-WITHDRAWN,manager,Y,163717.0,2016.0
12,CERTIFIED-WITHDRAWN,manager,Y,203860.8,2016.0


In [29]:
x = df.iloc[:,1:].values
y = df["CASE_STATUS"].values

In [30]:
x

array([['manager', 'Y', 242674.0, 2016.0],
       ['manager', 'Y', 220314.0, 2016.0],
       ['manager', 'Y', 157518.4, 2016.0],
       ...,
       ['others', 'Y', 33800.0, 2011.0],
       ['others', 'Y', 59800.0, 2011.0],
       ['others', 'Y', 59800.0, 2011.0]], dtype=object)

In [31]:
x.shape

(340000, 4)

In [32]:
y

array(['CERTIFIED-WITHDRAWN', 'CERTIFIED-WITHDRAWN', 'WITHDRAWN', ...,
       'WITHDRAWN', 'WITHDRAWN', 'WITHDRAWN'], dtype=object)

In [33]:
y.shape

(340000,)

In [34]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [35]:
ct = ColumnTransformer([('oh',OneHotEncoder(),[0,1])],remainder='passthrough')
x = ct.fit_transform(x)

In [36]:
x = x.toarray()

In [37]:
x.shape

(340000, 14)

In [38]:
np.set_printoptions(suppress=True)
print(x)

[[     0.       0.       0.  ...      1.  242674.    2016. ]
 [     0.       0.       0.  ...      1.  220314.    2016. ]
 [     0.       0.       0.  ...      1.  157518.4   2016. ]
 ...
 [     0.       0.       0.  ...      1.   33800.    2011. ]
 [     0.       0.       0.  ...      1.   59800.    2011. ]
 [     0.       0.       0.  ...      1.   59800.    2011. ]]


In [39]:
y

array(['CERTIFIED-WITHDRAWN', 'CERTIFIED-WITHDRAWN', 'WITHDRAWN', ...,
       'WITHDRAWN', 'WITHDRAWN', 'WITHDRAWN'], dtype=object)

In [40]:
y = y.reshape(y.shape[0],1)

In [41]:
# df['CASE_STATUS'] = df['CASE_STATUS'].map({'CERTIFIED' : 0, 'CERTIFIED-WITHDRAWN' : 1, 'DENIED' : 2,'WITHDRAWN' : 3,'PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED' : 4,'REJECTED' : 5, 'INVALIDATED' : 6})

In [42]:
# y = df["CASE_STATUS"].values

In [43]:
cty = ColumnTransformer([('ohy',OneHotEncoder(),[0])],remainder='passthrough')
y = cty.fit_transform(y)

In [44]:
y.shape

(340000, 4)

In [45]:
y = y.toarray()

In [46]:
y

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

**Train and Test**

In [47]:
from sklearn.model_selection import train_test_split

In [48]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [49]:
x_train.shape

(272000, 14)

In [50]:
x_train

array([[    0.,     0.,     0., ...,     1., 54434.,  2015.],
       [    0.,     0.,     0., ...,     1., 58344.,  2011.],
       [    0.,     0.,     0., ...,     1., 74630.,  2013.],
       ...,
       [    0.,     0.,     0., ...,     1., 98675.,  2014.],
       [    0.,     0.,     0., ...,     1., 59155.,  2015.],
       [    0.,     0.,     0., ...,     1., 44387.,  2011.]])

In [51]:
x_test.shape

(68000, 14)

In [52]:
x_test

array([[    0.,     0.,     0., ...,     1., 53914.,  2013.],
       [    0.,     0.,     0., ...,     1., 81286.,  2012.],
       [    0.,     0.,     0., ...,     1., 65206.,  2013.],
       ...,
       [    0.,     0.,     0., ...,     1., 76502.,  2014.],
       [    0.,     0.,     0., ...,     0., 62754.,  2016.],
       [    0.,     0.,     0., ...,     1., 73133.,  2014.]])

In [53]:
# y_train.reshape(y_train.shape[0],1)

In [54]:
# y_test.reshape(y_test.shape[0],1)

In [55]:
y_train.shape

(272000, 4)

In [56]:
y_train

array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]])

In [57]:
y_test.shape

(68000, 4)

In [58]:
y_test

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]])

In [59]:
#Feature Scaling

In [ ]:
# from sklearn.preprocessing import StandardScaler

In [ ]:
# sc = StandardScaler()
# x_train = sc.fit_transform(x_train);
# x_test = sc.transform(x_test)

In [ ]:
# x_train

In [ ]:
# x_test

**ANN**

In [60]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [95]:
# model.add(Dense(500, activation='relu', input_dim=14))
# model.add(Dense(100, activation='relu'))
# model.add(Dense(50, activation='relu'))
# model.add(Dense(2, activation='softmax'))

In [106]:
model = Sequential()

In [107]:
model.add(Dense(units=14,kernel_initializer="random_uniform",activation="relu"))
model.add(Dense(units=28,kernel_initializer="random_uniform",activation="relu"))
model.add(Dense(units=15,kernel_initializer="random_uniform",activation="relu"))
model.add(Dense(units=4,activation='softmax'))

In [108]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [109]:
history = model.fit(x_train,y_train,batch_size=1000,epochs=500,validation_data=(x_test,y_test))

Epoch 1/500
272/272 [==============================] - 1s 3ms/step - loss: 1.9857 - accuracy: 0.2515 - val_loss: 1.3860 - val_accuracy: 0.2391
Epoch 2/500
272/272 [==============================] - 1s 2ms/step - loss: 1.3931 - accuracy: 0.2534 - val_loss: 1.3787 - val_accuracy: 0.2750
Epoch 3/500
272/272 [==============================] - 1s 2ms/step - loss: 1.3861 - accuracy: 0.2615 - val_loss: 1.3749 - val_accuracy: 0.2880
Epoch 4/500
272/272 [==============================] - 1s 2ms/step - loss: 1.3784 - accuracy: 0.2805 - val_loss: 1.3763 - val_accuracy: 0.2787
Epoch 5/500
272/272 [==============================] - 1s 2ms/step - loss: 1.3750 - accuracy: 0.2852 - val_loss: 1.3726 - val_accuracy: 0.2901
Epoch 6/500
272/272 [==============================] - 1s 2ms/step - loss: 1.3724 - accuracy: 0.2905 - val_loss: 1.3712 - val_accuracy: 0.2903
Epoch 7/500
272/272 [==============================] - 1s 2ms/step - loss: 1.3717 - accuracy: 0.2920 - val_loss: 1.3733 - val_accuracy: 0.2873

In [125]:
yp = model.predict(ct.transform([["database","Y",75000.0,2015.0]]))

In [126]:
yp

array([[0.31991032, 0.252552  , 0.13487628, 0.29266134]], dtype=float32)

In [127]:
index = yp.argmax()

In [128]:
prediction = {0 : 'CERTIFIED' , 1: 'CERTIFIED-WITHDRAWN' , 2: 'DENIED' , 3: 'WITHDRAWN'}

In [129]:
prediction[index]

'CERTIFIED'

In [130]:
df.head()

,CASE_STATUS,SOC_NAME,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR
1,CERTIFIED-WITHDRAWN,manager,Y,242674.0,2016.0
3,CERTIFIED-WITHDRAWN,manager,Y,220314.0,2016.0
4,WITHDRAWN,manager,Y,157518.4,2016.0
11,CERTIFIED-WITHDRAWN,manager,Y,163717.0,2016.0
12,CERTIFIED-WITHDRAWN,manager,Y,203860.8,2016.0


In [131]:
# model.save("VisaPredModel.h5")

In [132]:
import joblib
# joblib.dump(ct,'columntransform')

['columntransform']